In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import cv2
import random
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt


In [ ]:

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load dataset
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)


In [ ]:
# Split dataset into train and test sets
def split_dataset(dataset, split_ratio=0.8):
    data = list(dataset)
    random.shuffle(data)
    split_index = int(len(data) * split_ratio)
    return data[:split_index], data[split_index:]

train_data, test_data = split_dataset(dataset['train'])


In [ ]:
# Function to preprocess images
def preprocess_image(image, label, img_size=(64, 64)):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, (-1,))  # Flatten the image
    return image.numpy(), label.numpy()

In [ ]:
# Prepare data for SVM
def prepare_data(data):
    images = []
    labels = []
    for image, label in data:
        img, lbl = preprocess_image(image, label)
        images.append(img)
        labels.append(lbl)
    return np.array(images), np.array(labels)

train_images, train_labels = prepare_data(train_data)
test_images, test_labels = prepare_data(test_data)

In [ ]:
# Train SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(train_images, train_labels)

# Evaluate model
test_predictions = svm_model.predict(test_images)
print("Classification Report:\n", classification_report(test_labels, test_predictions))
print("Accuracy: ", accuracy_score(test_labels, test_predictions))

In [ ]:
# Display random images
def display_random_images(data, num_images=16):
    random_images = random.sample(data, num_images)

    num_rows = num_images // 4
    num_cols = 4

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 5))
    axes = axes.ravel()

    for i, (image, label) in enumerate(random_images):
        image, _ = preprocess_image(image, label, img_size=(64, 64))
        image = image.reshape((64, 64, 3))
        axes[i].imshow(image)
        axes[i].set_title("Cat" if label == 0 else "Dog", fontsize=15)
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

display_random_images(train_data)

In [ ]:
# Function to classify a single image
def classify_single_image(image_path, model, img_size=(64, 64)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, img_size)
    image = image.astype('float32') / 255.0
    image = image.reshape((-1,))
    prediction = model.predict([image])
    return prediction[0]


In [ ]:
# Example usage
image_path = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTxKxnutgxwkYUpIdxXyfenk4Xm0t8lZN05SqulcPwsDA&s'  # Replace with the path to your JPEG image
prediction = classify_single_image(image_path, svm_model)
if prediction == 0:
    print("The image is classified as: Cat")
else:
    print("The image is classified as: Dog")